In [21]:
import pandas as pd
import numpy as np
import plotly.express as px
import os
from IPython.display import display as dp
import sys
sys.path.append(os.path.abspath(".."))  
from config.db import get_table_entregas

In [22]:
df=get_table_entregas()

In [23]:
dp(df.head())

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
import joblib

# 1. Preprocesamiento (convertir categorías a variables dummy)
X = pd.get_dummies(df.drop(columns=["id", "tiempo_entrega_dias"]))
y = df["tiempo_entrega_dias"]

# 2. Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [24]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
import joblib

In [25]:
X= pd.get_dummies(df.drop(columns=['id','tiempo_entrega_dias']))
y=df['tiempo_entrega_dias']

X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2, random_state=42)#el 42 es un numero neutral pero puede ser cualquier otro numero

# 3. Entrenar un modelo Random Forest
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)


In [26]:
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 4. Hacer predicciones y evaluar
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f"MAE: {mae:.2f} días")

In [27]:
y_pred = model.predict(X_test)
mae= mean_absolute_error(y_test, y_pred)
print(f'Error Absoluto Promedio (MAE): {mae:.2f} dias')

# 5. Guardar modelo como archivo .pkl en /models
joblib.dump(model, "../models/modelo_entrega.pkl")
print("✅ Modelo guardado como modelo_entrega.pkl")

In [28]:
joblib.dump(model, "../models/delivery_model.pkl")
print("✅ Modelo guardado como delivery_model.pkl")

# 6. Crear una muestra de predicción
nueva_entrada = pd.DataFrame([{
    "distancia_km": 75,
    "clima": "lluvia",
    "proveedor": "C"
}])

# 7. Codificar igual que en entrenamiento
entrada_procesada = pd.get_dummies(nueva_entrada)
entrada_procesada = entrada_procesada.reindex(columns=X.columns, fill_value=0)

# 8. Cargar modelo y predecir
modelo_cargado = joblib.load("../models/modelo_entrega.pkl")
prediccion = modelo_cargado.predict(entrada_procesada)
print(f"Predicción: {prediccion[0]:.2f} días")

In [29]:
ni=pd.DataFrame([{
    'distancia_km':75,
    "clima":'lluvia',
    "proveedor": 'C'
}])

input_processed = pd.get_dummies(ni)
input_processed = input_processed.reindex(columns=X.columns, fill_value=0)

charged_model=joblib.load("../models/delivery_model.pkl")
prediction= charged_model.predict(input_processed)
print(f"Prediccion: {prediction[0]:.2f} dias")

In [30]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

y_pred = model.predict(X_test)

mae=mean_absolute_error(y_test, y_pred)
mse=mean_squared_error(y_test, y_pred)
rmse=np.sqrt(mse)
r2=r2_score(y_test,y_pred)


print(f"MAE:  {mae:.2f} días")
print(f"MSE:  {mse:.2f} dias cuadrados ")
print(f"RMSE: {rmse:.2f} días")
print(f"R²:   {r2:.4f}")

In [ ]:
"""
📌 ¿Qué significan?
MAE bajo → En promedio, el modelo no se equivoca mucho (todos los errores pesan igual).

MSE bajo → También indica buen desempeño, pero penaliza más los errores grandes porque eleva al cuadrado cada diferencia.

Si MAE y MSE son casi iguales, significa que no hay errores extremos en tus predicciones. ✅

RMSE bajo → Es la raíz del MSE, así que se expresa en las mismas unidades (en tu caso, días). Ayuda a interpretar el error en la escala original.

R² cercano a 1 → El modelo explica muy bien la variación de los datos. R² = 1 significa predicción perfecta.

R² ≈ 0 o negativo → El modelo no está aprendiendo nada útil, y es peor que simplemente predecir el promedio de los datos.
"""

In [ ]:
"""
🧠 ¿Por qué el MSE es menor que el MAE aquí?
Tal como analizamos antes:

Tu modelo tiene errores pequeños

Todos los errores están probablemente entre -1 y 1

Y cuando elevas al cuadrado fracciones (como 0.3), se hacen aún más pequeñas:

(0.3)^2 = 0.09, (0.5)^2 = 0.25 
Así que sí: es matemáticamente válido que el MSE te dé menos que el MAE si tus errores están por debajo de 1 en valor absoluto.

🎯 Interpretación:
Tu modelo:

✅ Predice con muy poca desviación (±0.35 días en promedio)
✅ Tiene un coeficiente R² > 0.92, lo cual es excelente
✅ Es lo suficientemente bueno para pasar a producción


"""